In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
eps=np.finfo(float).eps
from binarytree import tree,Node
from sklearn.metrics import classification_report, confusion_matrix ,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from operator import itemgetter
import copy

In [2]:
df=pd.read_csv("train.csv")

In [3]:
X =df.drop(['left','sales','salary'],axis=1)
X = pd.concat([X,pd.get_dummies(df['sales'], prefix='sales')],axis=1)
X = pd.concat([X,pd.get_dummies(df['salary'], prefix='salary')],axis=1)
y=df['left']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
orignalX=copy.deepcopy(X_train)
orignalY=copy.deepcopy(y_train)
testX=copy.deepcopy(X_test)
testY=copy.deepcopy(y_test)

In [5]:
def findParentEntropy(df):
    if df.empty==True:
        return
    classCol=df.keys()[-1]
    #yeh sirf header return karega
    resultValues=df[classCol].unique()
    #all ouput values in output column
    entropy=0
    for value in resultValues:
        fraction=df[classCol].value_counts()[value]/(len(df[classCol])+eps)
        entropy+=-fraction*np.log2(fraction+eps)
    return abs(entropy)
    
    

In [6]:
def findRelativeEntropy(df,attribute):
    if df.empty==True:
        return
    classCol= df.keys()[-1] 
    #last col assumed as result 
    #getting target class 
    resultValues=df[classCol].unique() 
    attributeNames=df[attribute].unique()
    entropy2 = 0
    for attr in attributeNames:
        entropy = 0
        for value in resultValues:
            #temp&hot with yes
            num = len(df[attribute][df[attribute]==attr][df[classCol]==value])
            #total hot
            den = len(df[attribute][df[attribute]==attr])
            fraction = num/(den+eps)
            entropy += -fraction*np.log2(fraction+eps)
        #entropy attribute/total*(entropy attribute_values)
        fraction2 = den/len(df)  
        entropy2 += -fraction2*entropy 
    return abs(entropy2)

In [7]:
def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)


In [8]:
def get_subtableSmaller(df, node,value):
    if df.empty==True:
        return
    return df[df[node] < value].reset_index(drop=True)

In [9]:
def get_subtableGreater(df, node,value):
    if df.empty==True:
        return
    return df[df[node] >= value].reset_index(drop=True)

In [10]:
def bestAttribute(df):
    if df.empty==True:
        return
    infoGain=[]
    for key in df.keys()[:-1]:
        infoGain.append(findParentEntropy(df)-findRelativeEntropy(df,key))
    return df.keys()[:-1][np.argmax(infoGain)]
    

In [11]:

df=pd.concat([X_train,y_train],axis=1)
splitpoint={}
numeric=['last_evaluation','satisfaction_level','average_montly_hours','time_spend_company','number_project']
for col in numeric:
    df.sort_values(col,inplace=True)
    prev=None;
    value=0;
    max1=0;
    print(col)
    for index,row in df.iterrows():
        if prev is not None:
            if prev['left'] != row['left']:
                mid=(float(prev[col])+float(row[col]))/2
                subtableLeft=get_subtableSmaller(df,col,mid)
                subtableRight=get_subtableGreater(df,col,mid)
                fraction=len(subtableLeft)/len(df)
                fraction1=len(subtableRight)/len(df)
                entropy1=findParentEntropy(df)
#                 print(subtableLeft)
#                 a=input()
                firstSubtable=0
                secondSubtable=0
                if subtableLeft.empty:
                    firstSubtable=0
                else:
                    firstSubtable=fraction*findParentEntropy(subtableLeft)
#                 print(subtableRight)
                if subtableRight.empty:
                    secondSubtable=0
                else:
                    secondSubtable=fraction1*findParentEntropy(subtableRight)
                    
                entropy2=firstSubtable+secondSubtable
                ig1=abs(entropy1)-abs(entropy2)

                if ig1>max1:
                    max1=ig1
                    value=mid
        prev=row
    splitpoint[col]=value 
    for index,row1 in X_train.iterrows():
        if float(row1[col])<value:
            X_train.at[index,col]=0
        else:
            X_train.at[index,col]=1


last_evaluation
satisfaction_level
average_montly_hours
time_spend_company
number_project


In [12]:
# print(X_train)
# print(value)
# print(max1)
df=pd.concat([X_train,y_train],axis=1)
#df

In [13]:
class node:
    def __init__(self,value,left=None,right=None):
        self.value=value
        self.left=left
        self.right=right
        self.positive=0
        self.negative=0

In [14]:
def buildTree(df):
    if len(df.columns)==1:
        return
    
    resultLabel= df.keys()[-1] 
    clValue,counts = np.unique(df[resultLabel],return_counts=True)
    attribute_name=bestAttribute(df)
#     attValue = np.unique(df[attribute_name])
    root=node(attribute_name)
    
    if len(counts)<=1:
        if clValue[0]==0:
            root.negative=counts[0]
        else:
            root.positive=counts[0]
    else:
        root.positive=counts[1]
        root.negative=counts[0]
            
            
#         for value in attValue:
#             print(type(value))
        
        subtable0 = get_subtable(df,attribute_name,0)
        subtable0 =subtable0.drop([attribute_name],axis=1)
        subtable1 = get_subtable(df,attribute_name,1)
        subtable1 =subtable1.drop([attribute_name],axis=1)
        clValue,countleft = np.unique(subtable0[resultLabel],return_counts=True)
        clValue1,countright = np.unique(subtable1[resultLabel],return_counts=True)
        if(len(countleft)<=1):
            pass
        else:
            root.left=buildTree(subtable0) #Calling the function recursively
        if(len(countright)<=1):
            pass
        else:
            root.right=buildTree(subtable1)
       
                   
    return root

In [15]:
# print(df)
root=buildTree(df)
print(root.value)

satisfaction_level


In [16]:
print(root.right.value)
print(root.right.left)
print(root.left.value)
print(root.left.left.value)
print(root.left.right.value)
print(root.positive,root.negative)

time_spend_company
number_project
last_evaluation
average_montly_hours
2158 6832


In [17]:
def testing(row1,root):
#     if root==None:
#         return 
    #print(root.value)
    if root.left==None and root.right==None:
        if root.positive > root.negative:
            y1.append(1)
        else:
            y1.append(0)
        return
#     for i in row1.keys():
#         if i==root.value:
    i=root.value
    if row1[i]==0:
#         print("000")
        if root.left==None:
            if root.positive > root.negative:
                y1.append(1)
                return
            else:
                y1.append(0)
                return
        else:
            testing(row1,root.left)
    else:
#         print("111")
        if root.right==None:
            if root.positive > root.negative:
                y1.append(1)
                return
            else:
                y1.append(0)
                return 
        else:
            testing(row1,root.right)
                    
                

In [18]:
#X_test
print(splitpoint)

{'last_evaluation': 0.58, 'satisfaction_level': 0.46499999999999997, 'average_montly_hours': 286.5, 'time_spend_company': 3.0, 'number_project': 2.5}


In [19]:
def testingData(df):
    for index,row in df.iterrows():
        testing(row,root)

y1=[] 
for col in numeric:
    value=splitpoint[col]
    for index,row1 in X_test.iterrows():
            if float(row1[col])<value:
                X_test.at[index,col]=0
            else:
                X_test.at[index,col]=1
X_test
testingData(X_test)  
print(len(y1))


2248


In [20]:
print(confusion_matrix(y_test, y1))  
print(accuracy_score(y_test,y1))
print(classification_report(y_test, y1)) 

[[1681   50]
 [ 219  298]]
0.880338078291815
              precision    recall  f1-score   support

           0       0.88      0.97      0.93      1731
           1       0.86      0.58      0.69       517

   micro avg       0.88      0.88      0.88      2248
   macro avg       0.87      0.77      0.81      2248
weighted avg       0.88      0.88      0.87      2248



In [21]:
classifier = DecisionTreeClassifier()
classifier.fit(orignalX, orignalY)  
y_pred = classifier.predict(testX)  
print(confusion_matrix(testY, y_pred))  
print(classification_report(testY, y_pred)) 
print(accuracy_score(testY,y_pred))

[[1701   30]
 [  27  490]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1731
           1       0.94      0.95      0.95       517

   micro avg       0.97      0.97      0.97      2248
   macro avg       0.96      0.97      0.96      2248
weighted avg       0.97      0.97      0.97      2248

0.974644128113879
